# 올웨더 포트폴리오 분석 - 실행용

## 📚 이 노트북에서 배울 내용
1. 한국 ETF로 올웨더 포트폴리오 만들기
2. 실제 시장 데이터로 수익률 계산하기
3. 리스크 지표 이해하기
4. 포트폴리오 성과 시각화하기

## 💡 올웨더 포트폴리오란?
레이 달리오가 제안한 **자산배분 전략**으로, 어떤 경제 상황에서도 안정적인 수익을 추구합니다.

---

## STEP 1: 라이브러리 불러오기

In [2]:
!pip install pykrx pandas plotly

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 15.4 MB/s  0:00:00
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ----------------- ---------------------- 6.8/15.8 MB 32.3 MB/s eta 0:00:01
   ---------------------------------------  15.5/15.8 MB 37.4 MB/s eta 0:00:01
   ---------------------------------------- 15.8/15.8 MB 34.3 MB/s  0:00:00
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   --------------------------------- ------ 8.4/9.9 MB 43.2 MB/s eta 0:00:01
   ---------------------------------------- 9.9/9.9 MB 38.5 MB/s  0:00:00

  Attempting uninstall: numpy

    Found existing installation: numpy 2.2.5

   ----------- ---------------------------- 2/7 [numpy]
    Uninstalling numpy-2.2.5:
   ----------- ---------------------------- 2/7 [numpy]
   ----------- ---------------------------- 2/7 [numpy]
   ----------- ---------------------------- 2/7 [numpy]
  

In [1]:
# 필수 라이브러리 설치 (처음 실행 시)
# !pip install pykrx pandas plotly

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pykrx import stock
from datetime import datetime

# 출력 옵션 설정
pd.options.display.float_format = '{:.2f}'.format

print("✅ 라이브러리 로딩 완료!")

✅ 라이브러리 로딩 완료!


## STEP 2: 포트폴리오 구성하기

**자산 배분:**
- 주식 30% (성장)
- 장기채권 40% (안정성)
- 중기채권 15%
- 원자재 7.5%
- 금 7.5%

In [2]:
portfolio_df = pd.DataFrame({
    '자산군': ['주식', '장기채권', '중기채권', '원자재', '금'],
    '비중(%)': [30, 40, 15, 7.5, 7.5],
    'ETF명': [
        'KODEX 200',
        'ACE 미국30년국채액티브', 
        'TIGER 미국채10년선물',
        'KODEX 골드선물(H)',
        'KODEX 골드선물(H)'
    ],
    '티커': ['133690', '411060', '356540', '130680', '261240']
})

print("📋 포트폴리오 구성")
print("=" * 70)
print(portfolio_df.to_string(index=False))
print(f"\n✅ 총 비중: {portfolio_df['비중(%)'].sum()}%")

📋 포트폴리오 구성
 자산군  비중(%)           ETF명     티커
  주식  30.00      KODEX 200 133690
장기채권  40.00 ACE 미국30년국채액티브 411060
중기채권  15.00 TIGER 미국채10년선물 356540
 원자재   7.50  KODEX 골드선물(H) 130680
   금   7.50  KODEX 골드선물(H) 261240

✅ 총 비중: 100.0%


## STEP 3: 자산 비중 시각화

In [3]:
fig = px.pie(
    portfolio_df,
    values='비중(%)',
    names='자산군',
    title='올웨더 포트폴리오 자산 배분',
    hole=0.3,
    color_discrete_sequence=px.colors.qualitative.Set3
)

fig.update_traces(
    textposition='inside',
    textinfo='percent+label'
)

fig.show()

## STEP 4: ETF 데이터 수집

⏰ **주의:** 5개 ETF 데이터를 가져오므로 10~15초 정도 소요됩니다.

In [4]:
# 분석 기간 설정 (원하는 기간으로 변경 가능)
start_date = '20230101'
end_date = '20260131'

print(f"📥 {start_date} ~ {end_date} 기간의 데이터를 가져옵니다...\n")

price_data = {}

for idx, row in portfolio_df.iterrows():
    ticker = row['티커']
    asset_name = row['자산군']
    
    print(f"  └ {asset_name} ({ticker}) 수집 중...", end=" ")
    
    try:
        df = stock.get_etf_ohlcv_by_date(start_date, end_date, ticker)
        price_data[asset_name] = df['종가']
        print(f"✅ 완료 ({len(df)}개)")
    except Exception as e:
        print(f"❌ 오류: {e}")

price_df = pd.DataFrame(price_data)

print(f"\n✅ 전체 데이터 수집 완료! (총 {len(price_df)}일)")
print("\n📊 최근 5일 종가:")
print(price_df.tail())

📥 20230101 ~ 20260131 기간의 데이터를 가져옵니다...

  └ 주식 (133690) 수집 중... ✅ 완료 (752개)
  └ 장기채권 (411060) 수집 중... ✅ 완료 (752개)
  └ 중기채권 (356540) 수집 중... ✅ 완료 (752개)
  └ 원자재 (130680) 수집 중... ✅ 완료 (752개)
  └ 금 (261240) 수집 중... ✅ 완료 (752개)

✅ 전체 데이터 수집 완료! (총 752일)

📊 최근 5일 종가:
                주식   장기채권    중기채권   원자재      금
날짜                                            
2026-01-26  163530  33380  104390  4080  14620
2026-01-27  165495  34205  104440  4020  14655
2026-01-28  165035  35315  104570  4165  14440
2026-01-29  164915  37890  104315  4240  14465
2026-01-30  163585  35440  104130  4250  14610


## STEP 5: 데이터 품질 확인

In [ ]:
print("🔍 데이터 품질 체크")
print("=" * 70)
print(f"기간: {price_df.index[0]} ~ {price_df.index[-1]}")
print(f"거래일수: {len(price_df)}일")
print(f"\n결측치 개수:")
print(price_df.isnull().sum())

if price_df.isnull().sum().sum() > 0:
    print("\n⚠️  결측치를 이전 값으로 채웁니다.")
    price_df = price_df.fillna(method='ffill')

print("\n✅ 데이터 준비 완료!")

🔍 데이터 품질 체크
기간: 2023-01-02 00:00:00 ~ 2026-01-30 00:00:00
거래일수: 752일

결측치 개수:
주식      0
장기채권    0
중기채권    0
원자재     0
금       0
dtype: int64

✅ 데이터 준비 완료!


## STEP 6: 월별 수익률 계산

**수익률 공식:**
```
수익률 = (이번 달 가격 - 지난 달 가격) / 지난 달 가격
```

In [ ]:
# 월말 가격 추출
print("📅 월말 가격 추출")
monthly_price = price_df.resample('M').last()
print(f"  일별 {len(price_df)}개 → 월별 {len(monthly_price)}개로 변환\n")

# 월별 수익률 계산
monthly_return = monthly_price.pct_change().fillna(0)

print("최근 3개월 월말 가격:")
print(monthly_price.tail(3))

print("\n최근 3개월 수익률 (%):")
print((monthly_return.tail(3) * 100).round(2))

📅 월말 가격 추출
  일별 752개 → 월별 37개로 변환

최근 3개월 월말 가격:
                주식   장기채권    중기채권   원자재      금
날짜                                            
2025-11-30  165000  27880  105095  3905  14975
2025-12-31  163025  29005  105255  3830  14580
2026-01-31  163585  35440  104130  4250  14610

최근 3개월 수익률 (%):
              주식  장기채권  중기채권   원자재     금
날짜                                      
2025-11-30  0.32  5.23 -2.72 -1.88  3.49
2025-12-31 -1.20  4.04  0.15 -1.92 -2.64
2026-01-31  0.34 22.19 -1.07 10.97  0.21


C:\Users\Admin\AppData\Local\Temp\ipykernel_15332\1752861806.py:3: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



## STEP 7: 포트폴리오 전체 수익률 계산

**가중 평균:** 각 자산 수익률 × 비중을 모두 더하기

In [ ]:
# 비중을 소수로 변환
weights = portfolio_df.set_index('자산군')['비중(%)'] / 100

print("⚖️  각 자산의 가중치:")
print(weights)

# 포트폴리오 수익률 계산
portfolio_return = (monthly_return * weights).sum(axis=1)

print("\n📊 포트폴리오 월별 수익률 (최근 6개월):")
recent_returns = portfolio_return.tail(6) * 100
for date, ret in recent_returns.items():
    emoji = "📈" if ret > 0 else "📉" if ret < 0 else "➡️"
    print(f"  {emoji} {date.strftime('%Y-%m')}: {ret:+.2f}%")

⚖️  각 자산의 가중치:
자산군
주식     0.30
장기채권   0.40
중기채권   0.15
원자재    0.07
금      0.07
Name: 비중(%), dtype: float64

📊 포트폴리오 월별 수익률 (최근 6개월):
  📈 2025-08: +1.07%
  📈 2025-09: +12.43%
  📈 2025-10: +0.63%
  📈 2025-11: +1.90%
  📈 2025-12: +0.94%
  📈 2026-01: +9.65%


## STEP 8: 누적 수익 계산 (복리 효과)

**복리:** 매월 수익을 계속 곱하기!

In [ ]:
# 누적 수익 계산
portfolio_value = (1 + portfolio_return).cumprod()

print("💎 포트폴리오 가치 변화")
print("=" * 70)
print(f"시작: {portfolio_value.index[0].strftime('%Y-%m')}")
print(f"종료: {portfolio_value.index[-1].strftime('%Y-%m')}")
print(f"\n초기 투자금: 1.0")
print(f"최종 가치: {portfolio_value.iloc[-1]:.4f}")
print(f"총 수익률: {(portfolio_value.iloc[-1] - 1) * 100:.2f}%")

# 연평균 수익률 (CAGR)
years = len(portfolio_value) / 12
cagr = (portfolio_value.iloc[-1] ** (1/years) - 1) * 100
print(f"연평균 수익률(CAGR): {cagr:.2f}%")

💎 포트폴리오 가치 변화
시작: 2023-01
종료: 2026-01

초기 투자금: 1.0
최종 가치: 2.2484
총 수익률: 124.84%
연평균 수익률(CAGR): 30.05%


## STEP 9: 포트폴리오 성과 시각화

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=portfolio_value.index,
    y=portfolio_value.values,
    mode='lines',
    name='포트폴리오 가치',
    line=dict(color='#2E86AB', width=3),
    fill='tozeroy',
    fillcolor='rgba(46, 134, 171, 0.1)'
))

fig.add_hline(
    y=1.0,
    line_dash="dash",
    line_color="gray",
    annotation_text="초기 투자금"
)

fig.update_layout(
    title='올웨더 포트폴리오 누적 수익률',
    xaxis_title='날짜',
    yaxis_title='가치 (초기=1.0)',
    template='plotly_white',
    height=500
)

fig.show()

## STEP 10: 개별 자산 성과 비교

In [ ]:
# 각 자산의 누적 수익
cumulative_returns = (1 + monthly_return).cumprod()

fig = go.Figure()

# 각 자산 라인
for col in cumulative_returns.columns:
    fig.add_trace(go.Scatter(
        x=cumulative_returns.index,
        y=cumulative_returns[col],
        mode='lines',
        name=col,
        line=dict(width=2)
    ))

# 포트폴리오 전체 (굵은 선)
fig.add_trace(go.Scatter(
    x=portfolio_value.index,
    y=portfolio_value.values,
    mode='lines',
    name='포트폴리오 전체',
    line=dict(color='black', width=4, dash='dash')
))

fig.update_layout(
    title='자산별 누적 수익률 비교',
    xaxis_title='날짜',
    yaxis_title='누적 수익 (초기=1.0)',
    template='plotly_white',
    height=600
)

fig.show()

# 최종 수익률 비교
final_returns = (cumulative_returns.iloc[-1] - 1) * 100
final_returns['포트폴리오'] = (portfolio_value.iloc[-1] - 1) * 100

print("\n📊 자산별 최종 수익률:")
print("=" * 70)
for asset, ret in final_returns.sort_values(ascending=False).items():
    print(f"{asset:20s}: {ret:+7.2f}%")


📊 자산별 최종 수익률:
장기채권                : +223.95%
주식                  : +149.79%
포트폴리오               : +124.84%
금                   :  +32.16%
중기채권                :   +9.04%
원자재                 :   -0.47%


## STEP 11: 리스크 지표 계산

In [ ]:
# 변동성 (연율화)
volatility = portfolio_return.std() * (12 ** 0.5) * 100

# 샤프 비율 (무위험 수익률 3% 가정)
risk_free_rate = 0.03
excess_return = cagr / 100 - risk_free_rate
sharpe_ratio = excess_return / (volatility / 100)

# 최대 낙폭 (MDD)
cummax = portfolio_value.cummax()
drawdown = (portfolio_value - cummax) / cummax
max_drawdown = drawdown.min() * 100
mdd_date = drawdown.idxmin()

print("📈 포트폴리오 성과 지표")
print("=" * 70)
print(f"연평균 수익률(CAGR):     {cagr:>8.2f}%")
print(f"연평균 변동성:           {volatility:>8.2f}%")
print(f"샤프 비율:              {sharpe_ratio:>8.2f}")
print(f"최대 낙폭(MDD):         {max_drawdown:>8.2f}%")
print(f"최대 낙폭 발생:          {mdd_date.strftime('%Y년 %m월')}")

print("\n💡 해석:")
print(f"  • 연평균 {cagr:.1f}% 수익")
print(f"  • 변동성 {volatility:.1f}% (수익률의 들쭉날쭉함)")

if sharpe_ratio > 1:
    evaluation = "훌륭함 👍"
elif sharpe_ratio > 0.5:
    evaluation = "괜찮음 😊"
else:
    evaluation = "보통 😐"
    
print(f"  • 샤프 비율 {sharpe_ratio:.2f} → 위험 대비 수익이 {evaluation}")
print(f"  • 최악의 경우 {abs(max_drawdown):.1f}%까지 하락")

📈 포트폴리오 성과 지표
연평균 수익률(CAGR):        30.05%
연평균 변동성:              10.36%
샤프 비율:                  2.61
최대 낙폭(MDD):            -0.90%
최대 낙폭 발생:          2025년 04월

💡 해석:
  • 연평균 30.1% 수익
  • 변동성 10.4% (수익률의 들쭉날쭉함)
  • 샤프 비율 2.61 → 위험 대비 수익이 훌륭함 👍
  • 최악의 경우 0.9%까지 하락


## STEP 12: 낙폭(Drawdown) 시각화

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=drawdown.index,
    y=drawdown.values * 100,
    mode='lines',
    name='Drawdown',
    fill='tozeroy',
    line=dict(color='red', width=2),
    fillcolor='rgba(255, 0, 0, 0.2)'
))

fig.update_layout(
    title='포트폴리오 낙폭(Drawdown) 추이',
    xaxis_title='날짜',
    yaxis_title='낙폭 (%)',
    template='plotly_white',
    height=400
)

fig.show()

## STEP 13: 월별 수익률 히트맵

In [ ]:
# 데이터 준비
monthly_ret_pct = portfolio_return * 100
monthly_ret_pct.index = pd.to_datetime(monthly_ret_pct.index)

heatmap_data = pd.DataFrame({
    '연도': monthly_ret_pct.index.year,
    '월': monthly_ret_pct.index.month,
    '수익률': monthly_ret_pct.values
})

heatmap_pivot = heatmap_data.pivot(index='연도', columns='월', values='수익률')

# 히트맵 생성
fig = go.Figure(data=go.Heatmap(
    z=heatmap_pivot.values,
    x=[f"{m}월" for m in heatmap_pivot.columns],
    y=heatmap_pivot.index,
    colorscale='RdYlGn',
    zmid=0,
    text=heatmap_pivot.values.round(2),
    texttemplate='%{text}%',
    textfont={"size": 10},
    colorbar=dict(title="수익률(%)")
))

fig.update_layout(
    title='월별 수익률 히트맵',
    xaxis_title='월',
    yaxis_title='연도',
    height=400
)

fig.show()

print("\n💡 색깔 의미:")
print("  🟢 초록색: 수익이 난 달")
print("  🔴 빨간색: 손실이 난 달")
print("  🟡 노란색: 거의 변동 없음")


💡 색깔 의미:
  🟢 초록색: 수익이 난 달
  🔴 빨간색: 손실이 난 달
  🟡 노란색: 거의 변동 없음


## 📊 분석 완료!

### 🎯 주요 학습 내용

1. **데이터 수집**: pykrx로 실제 시장 데이터 가져오기
2. **수익률 계산**: `pct_change()`로 간단하게 계산
3. **포트폴리오 구성**: 가중 평균으로 전체 수익률 산출
4. **복리 효과**: `cumprod()`로 누적 수익 계산
5. **리스크 관리**: 변동성, 샤프 비율, MDD 이해
6. **시각화**: plotly로 인터랙티브 차트 생성

### 🔧 응용하기

다음을 바꿔서 실험해보세요:
- `start_date`, `end_date`: 분석 기간 변경
- `비중(%)`: 자산 비중 조정
- `티커`: 다른 ETF로 교체

### 💬 질문이 있다면?

- 각 단계의 코드를 수정해가며 결과를 관찰해보세요
- 에러가 나면 에러 메시지를 잘 읽어보세요
- 궁금한 점은 질문하세요!

# 티커: '005930' (삼성전자), '005380' (현대차)
- 누적 수익률 비교
- 변동성 비교
- 월별/분기별 성과
- 업종 대표 주식 이해

# KODEX 200 vs KODEX KOSDAQ150
- 시장 지수 개념 이해
- 대형주 vs 중소형주 비교
- 위험-수익 관계

# 고배당 ETF 3종 비교
티커: ['251350', '269540', '458730']
# KODEX 코스닥150선물인버스, KODEX 고배당, KODEX 미국S&P500배당귀족
- 배당 수익률 추정
- 안정성 지표
- 은퇴 자금 시뮬레이션